In [2]:
!pip install gradio textblob spacy langdetect PyPDF2 pytesseract pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 26.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 MB 27.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 624.3/624.3 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.7/815.7 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 25.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 26.8 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=9f3595bcb93784d76d6292da15bb048b5e169dd9968236103f72c59bdb4cd91c
  Stored in directory: /Users/hiteshchowdary/Library/Caches/pip/wheels/c1/67/88/e844b5b022812e15a52e4eaa38a1e709e99f06f6639d7e3ba7
Successfully built langdetect
  Attempting uninstall: brotli
    Found existing installation

In [10]:
import os
import gradio as gr
from textblob import TextBlob
import spacy
from langdetect import detect
import PyPDF2
from PIL import Image
import pytesseract

In [8]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 30.1 MB/s eta 0:00:0000:010:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [12]:
# Load spaCy model
nlp = spacy.load("en_core_web_sm")

In [14]:
MAX_SIZE_MB = 250  # size limit

In [16]:
def extract_text(file):
    """Extract text from txt, pdf, or image file"""
    ext = os.path.splitext(file.name)[-1].lower()
    
    if ext == ".txt":
        with open(file.name, "r", encoding="utf-8", errors="ignore") as f:
            return f.read()
    
    elif ext == ".pdf":
        text = ""
        try:
            reader = PyPDF2.PdfReader(file.name)
            for page in reader.pages:
                text += page.extract_text() or ""
        except Exception as e:
            return f"Error reading PDF: {e}"
        return text
    
    elif ext in [".jpg", ".jpeg", ".png"]:
        try:
            img = Image.open(file.name)
            return pytesseract.image_to_string(img)
        except Exception as e:
            return f"Error processing image: {e}"
    
    else:
        return "Unsupported file type. Please upload .txt, .pdf, .jpg, or .png"


In [20]:
def process_file(file):
    if file is None:
        return "No file uploaded."

    # Check size
    size_mb = os.path.getsize(file.name) / (1024 * 1024)
    if size_mb > MAX_SIZE_MB:
        return f"❌ File size {size_mb:.2f} MB > {MAX_SIZE_MB} MB. Not supportable."

    # Extract text
    text = extract_text(file)
    if not isinstance(text, str) or not text.strip():
        return "No readable text found."

    # Language detection
    try:
        lang = detect(text)
    except:
        lang = "Unknown"

    # Sentiment
    blob = TextBlob(text)
    sentiment = blob.sentiment

    # Keywords
    doc = nlp(text)
    keywords = [token.text for token in doc if token.pos_ in ["NOUN", "PROPN"]]
    keywords = list(set(keywords))[:15]

    # Entities
    entities = [(ent.text, ent.label_) for ent in doc.ents]

    # Summary (first 3 sentences)
    summary = " ".join([sent.text for sent in doc.sents][:3])

    return f"""
    File size: {size_mb:.2f} MB  
    **Language:** {lang}  
    **Sentiment (polarity):** {sentiment.polarity:.2f}  
    **Keywords (sample):** {keywords}  
    **Entities:** {entities}  
    **Summary:** {summary}
    """


In [22]:
# Gradio app
demo = gr.Interface(
    fn=process_file,
    inputs=gr.File(file_types=[".txt", ".pdf", ".jpg", ".jpeg", ".png"], label="Upload a File"),
    outputs="markdown",
    title="Multi-format NLP Application (with 250MB limit)"
)

demo.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
